In [1]:
from pathlib import Path
dataset_files = list(Path("Datasets/jsfakes-tok/bpe/").glob("**/*.json"))


In [6]:
import json
import pandas as pd
idList = []
for file in dataset_files:
    with open (file,"rb") as f:
        f_dict = json.load(f)
        idList.append(list(f_dict['ids']))
df = pd.DataFrame(columns=["train"])
df["train"]=idList


TypeError: the JSON object must be str, bytes or bytearray, not list

In [5]:
df.to_csv("Datasets/jsfakes-bpe.csv")

In [3]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="Datasets/jsfakes-bpe.csv")

# Convert dataset to dataframe
ds = dataset["train"]
df = ds.to_pandas()

In [4]:
# Take some sample from the dataset
raw_datasets = ds.train_test_split(test_size=0.1, shuffle=True)
sample= raw_datasets["train"][0]
sample

{'Unnamed: 0': 1889,
 'train': '[1700, 9721, 9835, 935, 4149, 4528, 1341, 5269, 577, 43, 2648, 2952, 3477, 1304, 2894, 38, 3253, 1585, 570, 3253, 577, 38, 3666, 391, 732, 3691, 570, 3028, 655, 40, 1341, 378, 8936, 710, 43, 742, 3573, 45, 742, 2789, 46, 221, 5154, 739, 1116, 1517, 3086, 4266, 262, 688, 3464, 2833, 3269, 9324, 1744, 55, 739, 3253, 2266, 1703, 378, 287, 1388, 1287, 41, 508, 739, 254, 629, 64, 516, 1899, 1198, 1675, 242, 2003, 50, 242, 789, 44, 311, 6012, 47, 4533, 664, 47, 2648, 391, 577, 50, 1157, 1509, 2963, 2047, 1084, 3922, 1671, 3984, 557, 1675, 1084, 2968, 1494, 4219, 1084, 5320, 8348, 1478, 33, 6962, 3795, 4453, 6229, 1606, 59, 159, 9850, 3984, 9725]'}

In [5]:
sample["train"]

'[1700, 9721, 9835, 935, 4149, 4528, 1341, 5269, 577, 43, 2648, 2952, 3477, 1304, 2894, 38, 3253, 1585, 570, 3253, 577, 38, 3666, 391, 732, 3691, 570, 3028, 655, 40, 1341, 378, 8936, 710, 43, 742, 3573, 45, 742, 2789, 46, 221, 5154, 739, 1116, 1517, 3086, 4266, 262, 688, 3464, 2833, 3269, 9324, 1744, 55, 739, 3253, 2266, 1703, 378, 287, 1388, 1287, 41, 508, 739, 254, 629, 64, 516, 1899, 1198, 1675, 242, 2003, 50, 242, 789, 44, 311, 6012, 47, 4533, 664, 47, 2648, 391, 577, 50, 1157, 1509, 2963, 2047, 1084, 3922, 1671, 3984, 557, 1675, 1084, 2968, 1494, 4219, 1084, 5320, 8348, 1478, 33, 6962, 3795, 4453, 6229, 1606, 59, 159, 9850, 3984, 9725]'

In [6]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel

# We need to specify the UNK token
new_tokenizer = Tokenizer(model=WordLevel(unk_token="[UNK]"))

# Add pretokenizer
from tokenizers.pre_tokenizers import WhitespaceSplit

new_tokenizer.pre_tokenizer = WhitespaceSplit()

# Let's test our pre_tokenizer
new_tokenizer.pre_tokenizer.pre_tokenize_str(sample["train"])




[('[1700,', (0, 6)),
 ('9721,', (7, 12)),
 ('9835,', (13, 18)),
 ('935,', (19, 23)),
 ('4149,', (24, 29)),
 ('4528,', (30, 35)),
 ('1341,', (36, 41)),
 ('5269,', (42, 47)),
 ('577,', (48, 52)),
 ('43,', (53, 56)),
 ('2648,', (57, 62)),
 ('2952,', (63, 68)),
 ('3477,', (69, 74)),
 ('1304,', (75, 80)),
 ('2894,', (81, 86)),
 ('38,', (87, 90)),
 ('3253,', (91, 96)),
 ('1585,', (97, 102)),
 ('570,', (103, 107)),
 ('3253,', (108, 113)),
 ('577,', (114, 118)),
 ('38,', (119, 122)),
 ('3666,', (123, 128)),
 ('391,', (129, 133)),
 ('732,', (134, 138)),
 ('3691,', (139, 144)),
 ('570,', (145, 149)),
 ('3028,', (150, 155)),
 ('655,', (156, 160)),
 ('40,', (161, 164)),
 ('1341,', (165, 170)),
 ('378,', (171, 175)),
 ('8936,', (176, 181)),
 ('710,', (182, 186)),
 ('43,', (187, 190)),
 ('742,', (191, 195)),
 ('3573,', (196, 201)),
 ('45,', (202, 205)),
 ('742,', (206, 210)),
 ('2789,', (211, 216)),
 ('46,', (217, 220)),
 ('221,', (221, 225)),
 ('5154,', (226, 231)),
 ('739,', (232, 236)),
 ('1116,'

In [9]:
# Yield batches of 1,000 texts
def get_training_corpus():
  dataset = raw_datasets["train"]
  for i in range(0, len(dataset), 1000):
    yield dataset[i : i + 1000]["text"]

from tokenizers.trainers import WordLevelTrainer

# Add special tokens
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

# Post-processing
new_tokenizer.save("tokenizer.json")